In [1]:
from imblearn.over_sampling import SMOTE  
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import os
from collections import Counter


In [2]:
# Chargement de la dataset
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    images, labels = [], []
    for label_dir in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label_dir)):
            img_path = os.path.join(dataset_path, label_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, img_size)
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)


In [3]:
images, labels = load_images_and_labels("./wheat_leaf")
images = images / 255.0  # Normalisation des images


In [4]:
# Encodage des étiquettes
lb = LabelBinarizer()
labels = lb.fit_transform(labels)



In [5]:
# Division en ensembles train, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [6]:
# Vérification de la distribution des classes dans le training set
class_distribution = Counter(np.argmax(y_train, axis=1))
print("Distribution des classes dans le training set :", class_distribution)


Distribution des classes dans le training set : Counter({2: 150, 1: 70, 0: 64})


In [7]:
# Application de SMOTE pour équilibrer les classes
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Aplatir les données pour SMOTE
smote = SMOTE(random_state=42)
X_augmented, y_augmented = smote.fit_resample(X_train_flattened, np.argmax(y_train, axis=1))


In [8]:
# Reshape des données augmentées
X_augmented = X_augmented.reshape(-1, 224, 224, 3)


In [9]:
from keras.utils import to_categorical

# Convertir y_augmented en one-hot encoding
y_augmented_one_hot = to_categorical(y_augmented, num_classes=3)

# Fusionner les données (X) et les étiquettes (y)
X_augmented_train = np.vstack((X_train, X_augmented))
y_augmented_train = np.vstack((y_train, y_augmented_one_hot))

print("Shape of X_augmented_train:", X_augmented_train.shape)
print("Shape of y_augmented_train:", y_augmented_train.shape)



Shape of X_augmented_train: (734, 224, 224, 3)
Shape of y_augmented_train: (734, 3)


### _________________________________________________ Xception _________________________________________________________

In [10]:
from tensorflow.keras.applications import Xception

# Construction du modèle MobileNetV2
Xception_model = Xception(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
Xception_model.trainable = False


In [11]:
model = models.Sequential([
    Xception_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


In [12]:
# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [13]:
# Entraînement du modèle
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 151s 6s/step - accuracy: 0.6106 - loss: 0.7476 - val_accuracy: 0.8361 - val_loss: 0.3785
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 133s 6s/step - accuracy: 0.9295 - loss: 0.2124 - val_accuracy: 0.8525 - val_loss: 0.3482
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 148s 6s/step - accuracy: 0.9743 - loss: 0.0937 - val_accuracy: 0.9180 - val_loss: 0.2373
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 139s 6s/step - accuracy: 0.9848 - loss: 0.0700 - val_accuracy: 0.8852 - val_loss: 0.2439
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 118s 5s/step - accuracy: 0.9991 - loss: 0.0339 - val_accuracy: 0.8361 - val_loss: 0.3619
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 138s 5s/step - accuracy: 1.0000 - loss: 0.0274 - val_accuracy: 0.9016 - val_loss: 0.2457
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step - accuracy: 1.0000 - loss: 0.0172 - val_accuracy: 0.8852 - val_loss: 0.3165
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 143s 6s/step - accuracy: 1.0000 - loss: 0.0132 - val_accuracy: 0.9016 - v

In [14]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.9788 - loss: 0.1208
Précision sur l'ensemble test : 0.98


In [15]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 16s 8s/step


In [16]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")
target_names = [str(cls) for cls in lb.classes_]
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[18  0  0]
 [ 0 13  0]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.95      1.00      0.97        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       1.00      0.97      0.98        31

    accuracy                           0.98        62
   macro avg       0.98      0.99      0.99        62
weighted avg       0.98      0.98      0.98        62



### ____________________________ MobileNetV2 ___________________________________________

In [17]:
from tensorflow.keras.applications import MobileNetV2

# Construction du modèle MobileNetV2
MobileNetV2_model = MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
MobileNetV2_model.trainable = False

In [19]:
model = models.Sequential([
    MobileNetV2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])


In [20]:
# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [21]:
# Entraînement du modèle
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 930ms/step - accuracy: 0.6978 - loss: 0.5854 - val_accuracy: 0.8852 - val_loss: 0.2925
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 651ms/step - accuracy: 0.9858 - loss: 0.0621 - val_accuracy: 0.8689 - val_loss: 0.2711
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 571ms/step - accuracy: 0.9843 - loss: 0.0429 - val_accuracy: 0.9508 - val_loss: 0.1715
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 542ms/step - accuracy: 0.9975 - loss: 0.0125 - val_accuracy: 0.9344 - val_loss: 0.1765
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 558ms/step - accuracy: 1.0000 - loss: 0.0070 - val_accuracy: 0.9180 - val_loss: 0.1810
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 24s 653ms/step - accuracy: 1.0000 - loss: 0.0049 - val_accuracy: 0.9180 - val_loss: 0.1934
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 545ms/step - accuracy: 1.0000 - loss: 0.0036 - val_accuracy: 0.9344 - val_loss: 0.1809
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 13s 543ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accu

In [22]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9150 - loss: 0.1649
Précision sur l'ensemble test : 0.92


In [23]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 6s/step


In [24]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")
target_names = [str(cls) for cls in lb.classes_]
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[15  0  3]
 [ 0 12  1]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.83      0.88        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.88      0.97      0.92        31

    accuracy                           0.92        62
   macro avg       0.94      0.91      0.92        62
weighted avg       0.92      0.92      0.92        62



### ______________________________________________ DenceNet121 __________________________________________

In [25]:

from tensorflow.keras.applications import DenseNet121

# Construction du modèle MobileNetV2
DenseNet121_model = DenseNet121(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet121_model.trainable = False

model = models.Sequential([
    DenseNet121_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])



In [26]:

# Entraînement du modèle
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 110s 3s/step - accuracy: 0.6852 - loss: 0.6990 - val_accuracy: 0.9016 - val_loss: 0.3084
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.9359 - loss: 0.1688 - val_accuracy: 0.9672 - val_loss: 0.1935
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9856 - loss: 0.0815 - val_accuracy: 0.9180 - val_loss: 0.1801
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.9995 - loss: 0.0394 - val_accuracy: 0.9836 - val_loss: 0.1290
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 1.0000 - loss: 0.0247 - val_accuracy: 0.9836 - val_loss: 0.1187
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 1.0000 - loss: 0.0195 - val_accuracy: 0.9836 - val_loss: 0.1188
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 1.0000 - loss: 0.0159 - val_accuracy: 0.9836 - val_loss: 0.1110
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 1.0000 - loss: 0.0104 - val_accuracy: 0.9836 - val_loss

In [27]:

# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")



2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9785 - loss: 0.0427
Précision sur l'ensemble test : 0.97


In [28]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


1/2 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001890A5C7560> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 31s 15s/step


In [29]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")
target_names = [str(cls) for cls in lb.classes_]
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))

Matrice de confusion :
[[18  0  0]
 [ 0 11  2]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00        18
    septoria       1.00      0.85      0.92        13
 stripe_rust       0.94      1.00      0.97        31

    accuracy                           0.97        62
   macro avg       0.98      0.95      0.96        62
weighted avg       0.97      0.97      0.97        62



### ________________________________________ DenseNet169 ___________________________________________________

In [30]:
from tensorflow.keras.applications import DenseNet169

# Construction du modèle MobileNetV2
DenseNet169_model = DenseNet169(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet169_model.trainable = False

model = models.Sequential([
    DenseNet169_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])



In [31]:
# Entraînement du modèle
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 193s 5s/step - accuracy: 0.6506 - loss: 0.7964 - val_accuracy: 0.8197 - val_loss: 0.3906
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.9438 - loss: 0.1499 - val_accuracy: 0.9180 - val_loss: 0.2425
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 72s 3s/step - accuracy: 0.9848 - loss: 0.0623 - val_accuracy: 0.9180 - val_loss: 0.2384
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 74s 3s/step - accuracy: 1.0000 - loss: 0.0309 - val_accuracy: 0.9344 - val_loss: 0.1650
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 64s 3s/step - accuracy: 1.0000 - loss: 0.0190 - val_accuracy: 0.9180 - val_loss: 0.2333
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 1.0000 - loss: 0.0159 - val_accuracy: 0.9344 - val_loss: 0.2150
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0100 - val_accuracy: 0.9344 - val_loss: 0.1831
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 1.0000 - loss: 0.0080 - val_accuracy: 0.9344 - val_loss

In [32]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9577 - loss: 0.1091
Précision sur l'ensemble test : 0.97


In [33]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 47s 22s/step


In [34]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")
target_names = [str(cls) for cls in lb.classes_]
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[16  0  2]
 [ 0 13  0]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      0.89      0.94        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       0.94      1.00      0.97        31

    accuracy                           0.97        62
   macro avg       0.98      0.96      0.97        62
weighted avg       0.97      0.97      0.97        62



### ____________________________________________ ResNet50V2 __________________________________________

In [35]:
from tensorflow.keras.applications import ResNet50V2
# Construction du modèle MobileNetV2
ResNet50V2_model = ResNet50V2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
ResNet50V2_model.trainable = False

model = models.Sequential([
    ResNet50V2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])




In [36]:
# Entraînement du modèle
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.7191 - loss: 0.6685 - val_accuracy: 0.8361 - val_loss: 0.3646
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9814 - loss: 0.0634 - val_accuracy: 0.9016 - val_loss: 0.2464
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 1.0000 - loss: 0.0148 - val_accuracy: 0.8689 - val_loss: 0.2832
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 1.0000 - loss: 0.0073 - val_accuracy: 0.8852 - val_loss: 0.2701
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 0.8689 - val_loss: 0.2872
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 56s 2s/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.8852 - val_loss: 0.2797
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 57s 3s/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.8852 - val_loss: 0.2820
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 55s 2s/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.8852 - val_loss:

In [37]:

# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9362 - loss: 0.0972
Précision sur l'ensemble test : 0.94


In [38]:

# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 9s/step


In [39]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

print("Rapport de classification :")
target_names = [str(cls) for cls in lb.classes_]
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))

Matrice de confusion :
[[16  0  2]
 [ 0 12  1]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.89      0.91        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.91      0.97      0.94        31

    accuracy                           0.94        62
   macro avg       0.95      0.93      0.94        62
weighted avg       0.94      0.94      0.94        62

